In [5]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
import openai
from dotenv import load_dotenv
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache, set_debug
import json
from pathlib import Path
from langchain.memory import ConversationSummaryBufferMemory
from langchain_community.document_loaders import JSONLoader
load_dotenv()
set_llm_cache(SQLiteCache("cache.db"))

class Chatbot:
    def __init__(self):

        #llm 모델 정의              
        self.llm = ChatOpenAI(
            temperature=0.5,
            streaming=True,
            model_name="gpt-4",
        )

        #프롬프트 정의
        self.prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """
                    You are a teacher who specializes in counseling with parents.
                    You should never say the details of the consultation as they are. Instead, you should organize the consultation.
                    You need to consult like a counselor
                    Parents' questions must be answered based on the counseling journal.
                    You have to answer in the same language as the questioner.
                    If it is a question that is not related to the student, it should be said that you cannot answer it.
                    When you answer, make sure to include the supporting counseling information.
                    If you don't know, or if you can't answer your question within the consultation details, please tell me that the relevant consultation has not been conducted
                    If human refers to "me" it means parents.
                    I'll give you an example based on the consultation details
                    If your child asks you in Korean, please answer in the following format
                    {{
                    "studentName": "최준원",
                    "studentNum": "010-4599-3761",
                    "studentParentNum": "010-4741-3761",
                    "consultations": [
                        {{
                        "date": "2024-08-13",
                        "method": "visit",
                        "client": "parent",
                        "location": "학교",
                        "category": "School life",
                        "contents": "집에서 아이가 학교에 가기 싫어하는 것을 느끼고 있어 학부모님이 걱정을 한다.
                                    학생과의 면담을 진행하여 이를 해결해보겠다."
                        }},
                        {{
                        "date": "2024-08-12",
                        "method": "visit",
                        "client": "student",
                        "location": "학교",
                        "category": "School life",
                        "contents": "우울한 기분이 자주 들며, 친구들과의 교류가 줄어듦.정서적 지원을 받을 수 있는 방법과 전문가 상담의 필요성을 논의."
                        }},
                    ]
                    }}
                    human: "How is your school life these days?"
                    you:" You asked about student Choi Joon-won's school life.
                        Choi Joon-won feels depressed and feels less interaction with his friends during the consultation date on Aug. 12. Also, parents are worried because they tend not to want to go to school at home during the consultation on Aug. 12.
                        To sum up, Choi Joon-won has recently become estranged from his friends due to his depressed feelings and does not want to go to school. Therefore, he is trying to conduct an interview with the student with the help of a professional."
                    
                    Let's start counseling now
                    Context:{details}
                    """,
                ),
                MessagesPlaceholder(variable_name="history"),
                ("human", "{question}"),
            ]
        )


        self.file_path = './1.json'
        self.data = Path(self.file_path).read_text(encoding='utf-8')
        #메모리 추가
        self.memory = ConversationSummaryBufferMemory(
            llm=self.llm,
            max_token_limit=450,
            return_messages=True,
            )
        
        #질문을 처리하는 llm chain
        self.questions_chain = (
            # 먼저 history를 로드함
            RunnablePassthrough.assign(history=self.load_memory)
            # 그런 다음 details, history, question을 함께 전달
            | RunnableLambda(lambda vars: {
                "details": json.loads(self.data),
                "history": vars["history"],
                "question": vars["question"]
            })
            | self.prompt
            | self.llm
        )


    
    #메모리를 context에 추가하도록함
    def load_memory(self,_):
        return self.memory.load_memory_variables({})["history"]

    #이 함수만 사용하면 됨
    # 두 체인을 연결하여 질문을 처리하고 json형식으로 나오도록함
    def invoke_chain(self,question):
        response = self.questions_chain.invoke({"question": question}).content
        self.memory.save_context(
            {"human": question},
            {"You": response},
        )
        # JSON 응답 생성
        response_json = {"message": response}
        for token in self.questions_chain.stream({"question":question}):
            print(token.content,end="")


        

In [6]:

chatbot=Chatbot()
chatbot.invoke_chain("요점 학교생활은 어때??")





"김일번 학생의 학교 생활에 대해 문의하셨군요. 
2024년 8월 2일 상담에서 김일번 학생은 운동 부족으로 인해 몸이 무겁게 느껴진다고 말씀하셨습니다. 그에 따라 짧은 시간이라도 운동을 시작하는 습관을 들일 것을 권장하였습니다.
요약하자면, 김일번 학생은 현재 운동 부족으로 인한 불편함을 겪고 있으며, 이를 해결하기 위해 운동을 시작하는 것이 필요하다는 것을 알 수 있습니다."